# Capstone Project

In [139]:
import requests
import pandas as pd
import numpy as np
import sqlite3 as lite
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import time
import datetime
from dateutil.parser import parse
import collections
import json
import ijson
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics

## Try some tactics to deal with huge file

In [12]:
# %%bash
# head -1 yelp_academic_dataset_review.json

In [13]:
# %%bash
# head -1 yelp_academic_dataset_review.json > output.txt

In [14]:
# %%bash
# echo -e output.csv

In [15]:
# %%bash
# cat yelp_academic_dataset_review.json | cut -d \, -f 6 > stars.csv

In [16]:
# %%bash
# cat yelp_academic_dataset_review.json | cut -d \, -f 8 > review.txt

In [17]:
# %%bash
# cat yelp_academic_dataset_review.json | cut -d \: -f 10 > review.csv

In [18]:
# %%bash

# grep -e '^{"' yelp_academic_dataset_review.json

In [19]:
# with open('yelp_academic_dataset_review.json','rU') as f:
#     data = json.load(f)

In [20]:
# for prefix, the_type, value in ijson.parse(open('yelp_academic_dataset_review.json')):
# #     if the_type == 'map_key':
# #         if value == 'stars':
# #         print the_type, value
#     print prefix, the_type, value

In [21]:
# df = pd.read_csv('stars.csv',names=['Stars','Text'])

In [22]:
# df['Stars'] = df['Stars'].map(lambda x: str(x)[-1])

In [23]:
# df

In [24]:
# df2 = pd.read_csv('review.csv')

In [25]:
# parser = ijson.parse(open('yelp_academic_dataset_review.json'))
# reviews = []
# for prefix, event, value in parser:
#     if (prefix, event) == ('start_map','map_key'):
#         reviews.append(value)
# # stream.write('</yelp>')
# reviews

In [26]:
# def load_json(filename):
#     with open(filename, 'r') as fd:
#         parser = ijson.parse(fd)
#         ret = {'stars':{}}
#         for prefix, event, value in parser:
#             if (prefix, event) == ('stars', 'map_key'):
#                 star_amount = value
#                 ret['stars'][star_amount] = {}
#         return ret

In [27]:
# df = pd.DataFrame(columns=['stars','text'])

# with open('yelp_academic_dataset_review.json','rU') as f:
#     for line in f:
#         data = json.loads(line)
        
# result = json_normalize(data, ['stars', 'text'])
# # df['stars'] = data['stars']
# # df['text'] = data['text']

## Iteratively load json's line by line

In [ ]:
# data = []
# for line in open('yelp_academic_dataset_review.json','rU'):
#     data.append(json.loads(line))
# data

In [29]:
data = []
with open('yelp_academic_dataset_review.json','rU') as reviews:
#     i = 0
#     while i < 10000:
    
    first_line = json.loads(reviews.readline())
    print (first_line['stars'], first_line['text'])
#         i += 1


(4, u'Mr Hoagie is an institution. Walking in, it does seem like a throwback to 30 years ago, old fashioned menu board, booths out of the 70s, and a large selection of food. Their speciality is the Italian Hoagie, and it is voted the best in the area year after year. I usually order the burger, while the patties are obviously cooked from frozen, all of the other ingredients are very fresh. Overall, its a good alternative to Subway, which is down the road.')


## Create training set from first 10,000 reviews, testing set from the next 5,000 reviews:

In [145]:
# data = []
# with open('yelp_academic_dataset_review.json','rU') as reviews:
#     i = 0
#     while i < 10000:
#         first_line = json.loads(reviews.readline())
#         if first_line['stars'] != 3:
#             data.append([first_line['stars'], first_line['text']])
#         i += 1
# # data

In [158]:
data = []
data_test = []
with open('yelp_academic_dataset_review.json','rU') as reviews:
    i = 0
    while i < 10000:
        first_line = json.loads(reviews.readline())
        if first_line['stars'] != 3:
            if first_line['stars'] < 3:
                first_line['stars'] = 1
            else:
                first_line['stars'] = 5
            data.append([first_line['stars'], first_line['text']])
        i += 1
    while 9999 <= i < 15000:
        first_line = json.loads(reviews.readline())
        if first_line['stars'] != 3:
            if first_line['stars'] < 3:
                first_line['stars'] = 1
            else:
                first_line['stars'] = 5
            data_test.append([first_line['stars'], first_line['text']])
        i += 1

# data_test

## Sort reviews by star number, make dataframes:

In [147]:
df = pd.DataFrame(data, columns = ['stars','text']).sort('stars')
df.reset_index(drop=True, inplace=True)

In [148]:
df.head()

,stars,text
0,1,just got off the phone with a lady from constr...
1,1,everything is a cluster. the store. the park...
2,1,The preparation was fast and the price was ver...
3,1,MEAN CUSTOMERS -ALOOF WORKERS+GOOD FOOD = CRAZ...
4,1,This is by far the worst Chinese food I have e...


In [150]:
df_testing = pd.DataFrame(data_test, columns = ['stars','text']).sort('stars')
df_testing.reset_index(drop=True, inplace=True)

In [149]:
df_testing.head()

,stars,text
0,1,I can't properly review the food or beverage s...
1,1,Owner is rude and condescending\nOverpriced co...
2,1,The two stars are for the food only. Otherwise...
3,1,Should be called crappys. So disappointed in f...
4,1,The food was terrible. I would rather have eat...


## Clean text for Bag-of-Words:

In [34]:
example1 = BeautifulSoup(df['text'][75])
example1.get_text()

/Users/jacobmink/anaconda/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


u'STAY AWAY...\nWe\'ve been 3 times over the past few years, each time we say "let\'s try it again" followed by "wish we\'d done something else".\n\nReminds me of the movie line "Run away, run very far away"\n\nLast nite after Steelers/Jets was last straw. I hope the owner or general manager reads this. And think the health department should too.\n\nService is mediocre and slow. consistently. Waitress was ok in attitude.\n\nAfter our soup starter (good french onion soup) we waited and waited and waited and waited for our mains. Noticing us staring at her, waitress stopped by, went into kitchen, came back and said we were next. my watch said 8:31. Then her manager came by and argued with me and said we were next, then at 8:52pm the horrible food finally arrived. That\'s 21 minutes to be next. In that time we watched 4 tables turn from being seated to drinks to mains to check down while we waited for our main. \n\nManager embarrassed himself. Said they were busy and working our food and 

In [35]:
def review_to_words(raw_review):
    #     1) Remove HTML:
    review_text = BeautifulSoup(raw_review).get_text()
    #     2) Remove non-letters:
    letters_only = re.sub('[^a-zA-Z]',' ', review_text)
    #     3) Convert to lower case, split into words:
    words = letters_only.lower().split()
    #     4) Convert stopwords to set:
    stops = set(stopwords.words('english'))
    #     5) Remove stopwords:
    meaningful_words = [w for w in words if not w in stops]
    #     6) Join words back into one string separated by space:
    return( ' '.join(meaningful_words))

In [36]:
clean_example = review_to_words(df['text'][0])
clean_example

u'moment walked knew interesting experience instead greeted smiling friendly hi welcome cains greeted frantically working unhelpful stressed hostess clue deal influx people breakfast rush walked hostess told us would minutes kitchen prepare food since known cains delicious breakfast thought wait would worth hour fifteen minutes later get food breakfast burrito tasted like nothing food cold good thing experience warm donuts poor service crappy food overall way restaurant run'

In [37]:
num_reviews = df['text'].size
num_reviews

8429

In [60]:
num_test_reviews = df_testing['text'].size
num_test_reviews

4273

In [107]:
print "Cleaning and parsing the training set Yelp reviews...\n"
clean_train_reviews = []
for i in xrange(0, num_reviews):
    if (i+1)%1000 == 0:
        print "Review %d of %d\n" % (i+1,num_reviews)
    clean_train_reviews.append(review_to_words(df['text'][i]))

Cleaning and parsing the training set Yelp reviews...

Review 1000 of 8429

Review 2000 of 8429

Review 3000 of 8429

Review 4000 of 8429

Review 5000 of 8429

Review 6000 of 8429

Review 7000 of 8429

Review 8000 of 8429



In [93]:
print "Cleaning and parsing the testing set Yelp reviews...\n"
clean_test_reviews = []
for i in xrange(0, num_test_reviews):
    if (i+1)%1000 == 0:
        print "Review %d of %d\n" % (i+1,num_test_reviews)
    clean_test_reviews.append(review_to_words(df_testing['text'][i]))

Cleaning and parsing the testing set Yelp reviews...

Review 1000 of 4273

Review 2000 of 4273

Review 3000 of 4273

Review 4000 of 4273



In [152]:
clean_test_reviews[0]

u'properly review food beverage selections since never actually got seated came part large group specifically chose harris grill could handle crowd arrived lounge area clearly designed seat people occupied precisely two individuals right two people leisurely sipping drinks host informed us entered would leaving soon half hour later ordered another drink seemed making small talk host clearly friend begs question earth would host seat two people party section came two possible answers restaurant hates money hired worst host pittsburgh returning harris grill sometime month try food looked smelled amazing whisked past plates destined tables response comment wrote review call attention business practice seems mutually detrimental restaurant customer hold grudges willing treat experience one'

In [153]:
clean_train_reviews[0]

u'got phone lady construction junction normally best experiences cooperative helpful guess successful becoming arrogant rub hubby sent picture chandelier see picture minutes later left store called buy chandelier phone done construction junction junkies lady adamantly refused help adamantly refuesed even listen description fixture gave following excuses well last years know place works go back try find want one floor first come first serve husband comes still sell know one many back could hear smirk voice enjoyed blowing even offered send picture husband sent via email refused keep mind wanted pay item husband cannot go back construction junction today likely item gone would nice sales agent helpful especially clear badly wanted buy fixture express concern whatsoever requests like sorry help moment please call back try accomodate give brief description try find etc sometimes even accomodate someone say makes difference construction junction implode begins treat customers indifferently'

## Vectorize word counts, train on training set:

In [108]:
vectorizer = CountVectorizer(analyzer='word',
                             tokenizer=None,
                             preprocessor=None,
                             stop_words=None,
                             max_features=5000)

train_data_features = vectorizer.fit_transform(clean_train_reviews)
train_data_features = train_data_features.toarray()

## Vectorize testing set words:

In [97]:
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

In [102]:
test_data_features.shape

(4273, 5000)

In [101]:
train_data_features.shape

(8429, 5000)

In [154]:
vocab = vectorizer.get_feature_names()
vocab[1:20]

[u'aamco',
 u'ability',
 u'able',
 u'absolute',
 u'absolutely',
 u'ac',
 u'accent',
 u'accept',
 u'acceptable',
 u'accepted',
 u'access',
 u'accessible',
 u'accessories',
 u'accident',
 u'accidentally',
 u'accommodate',
 u'accommodated',
 u'accommodating',
 u'accomodate']

In [155]:
dist = np.sum(train_data_features, axis=0)
for tag, count in zip(vocab[1:20], dist):
    print count, tag

17 aamco
13 ability
18 able
381 absolute
59 absolutely
381 ac
20 accent
9 accept
37 acceptable
34 accepted
10 access
39 accessible
18 accessories
10 accident
29 accidentally
12 accommodate
35 accommodated
13 accommodating
78 accomodate


## Generate random forest model:

In [109]:
print 'Training the random forest...'

forest = RandomForestClassifier(n_estimators=100)
forest = forest.fit(train_data_features, df['stars'])

Training the random forest...


In [110]:
print 'Testing using the random forest...'
result = forest.predict(test_data_features)
output = pd.DataFrame(data={'text':df_testing['text'],'stars_pred':result})

Testing using the random forest...


In [111]:
output

,stars_pred,text
0,1,I can't properly review the food or beverage s...
1,1,Owner is rude and condescending\nOverpriced co...
2,1,The two stars are for the food only. Otherwise...
3,1,Should be called crappys. So disappointed in f...
4,1,The food was terrible. I would rather have eat...
5,1,I'd give this joint zero stars if it was possi...
6,1,Horrible service. Waited 50 minutes and had no...
7,1,Machines are dirty. Ruined my pants and then t...
8,5,"In my opinion, the is a prime example of a bad..."
9,1,So I don't know about the actual work of this ...


## Evaluate model predictions:

In [156]:
difference = result - df_testing['stars']

In [143]:
length = float(len(df_testing['stars']))

In [142]:
rmse = (sum((difference)**2)/length)**0.5
rmse

1.3052666543977181

In [136]:
mse = sum((difference)**2)/length
mse

1.7037210390826119

In [140]:
r2 = sklearn.metrics.r2_score(df_testing['stars'], result)


0.41126853965927612

In [141]:
r2**0.5

0.6413022217794635

In [144]:
print 'R-squared value for random forest model: {}'.format(r2)
print 'Mean Squared Error value: {}'.format(mse)
print 'Root Mean Squared Error value: {}'.format(rmse)

R-squared value for random forest model: 0.411268539659
Mean Squared Error value: 1.70372103908
Root Mean Squared Error value: 1.3052666544
